In [ ]:
!pip install -q catboost lightgbm xgboost imbalanced-learn scikit-learn tqdm shap joblib

import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_predict
from sklearn.preprocessing import MinMaxScaler
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, balanced_accuracy_score,
    roc_curve, auc
)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/UKBB 분석')


data = pd.read_csv("backpain_28639사용_결측처리(모두MICE).csv")
data.columns = data.columns.str.strip()

Mounted at /content/drive


In [ ]:
print(">>>> 데이터 로드 완료:", data.shape)
display(data.head())
data.info()

>>>> 데이터 로드 완료: (189401, 17)


,sex,ethnicity,education,income,occupation,social_activity,social_contacts_frequency,household_size,moderate_exercise_frequency,alcohol_frequency,sleep_hours,fatigue_frequency,feeling_depressed,feeling_anxious,low_back_pain,age,participant_id
0,0,0,2,3,0,1,4,1,2,3,7.29,1,0,0,1,84.0,1000020
1,1,0,0,1,0,1,4,1,2,2,7.17,1,0,0,0,80.0,1000199
2,0,0,2,1,0,1,4,1,1,2,8.00,4,2,2,1,76.0,1000894
3,1,0,2,1,0,1,4,1,0,3,7.00,1,0,0,1,82.0,1000925
4,0,0,4,2,1,1,3,1,2,3,7.32,1,0,0,0,60.0,1001066


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189401 entries, 0 to 189400
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   sex                          189401 non-null  int64  
 1   ethnicity                    189401 non-null  int64  
 2   education                    189401 non-null  int64  
 3   income                       189401 non-null  int64  
 4   occupation                   189401 non-null  int64  
 5   social_activity              189401 non-null  int64  
 6   social_contacts_frequency    189401 non-null  int64  
 7   household_size               189401 non-null  int64  
 8   moderate_exercise_frequency  189401 non-null  int64  
 9   alcohol_frequency            189401 non-null  int64  
 10  sleep_hours                  189401 non-null  float64
 11  fatigue_frequency            189401 non-null  int64  
 12  feeling_depressed            189401 non-null  int64  
 13 

In [ ]:
X = data.drop(columns=["low_back_pain", "participant_id"])
y = data["low_back_pain"]

In [ ]:
# 도메인별 변수 정의
bio_cols = ['sex', 'age', 'ethnicity']
psych_cols = ['fatigue_frequency', 'feeling_depressed', 'feeling_anxious']
social_cols = ['education', 'income', 'occupation',
               'social_activity', 'social_contacts_frequency', 'household_size']
lifestyle_cols = ['moderate_exercise_frequency', 'alcohol_frequency', 'sleep_hours']

domain_cols = {
    "Bio": bio_cols,
    "Psych": psych_cols,
    "Social": social_cols,
    "Lifestyle": lifestyle_cols
}

continuous_cols_global = ['age', 'sleep_hours']
domain_continuous = {
    d: [c for c in cols if c in continuous_cols_global]
    for d, cols in domain_cols.items()
}

In [ ]:
# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


In [ ]:
# 1) 하이퍼파라미터 후보 (RandomizedSearchCV용)
param_dists = {
    "Logistic Regression": {"clf__C": [0.01, 0.1, 1, 10]},
    "Random Forest": {
        "clf__n_estimators": [100, 200, 300, 500],
        "clf__max_depth": [5, 10, None],
        "clf__min_samples_split": [2, 5],
        "clf__min_samples_leaf": [1, 2]
    },
    "XGBoost": {
        "clf__n_estimators": [100, 200, 300, 500],
        "clf__max_depth": [3, 5, 7],
        "clf__learning_rate": [0.05, 0.1, 0.2]
    },
    "LightGBM": {
        "clf__n_estimators": [100, 200, 300, 500],
        "clf__max_depth": [-1, 5, 10],
        "clf__learning_rate": [0.05, 0.1, 0.2]
    },
    "CatBoost": {
        "clf__n_estimators": [100, 200, 300, 500],
        "clf__depth": [5, 7, 9],
        "clf__learning_rate": [0.05, 0.1, 0.2]
    }
}

def make_base_clf(name):
    if name == "Logistic Regression":
        return LogisticRegression(max_iter=1000, solver='liblinear', random_state=2023)
    elif name == "Random Forest":
        return RandomForestClassifier(random_state=2023, n_jobs=-1)
    elif name == "XGBoost":
        return XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=2023, n_jobs=-1)
    elif name == "LightGBM":
        return LGBMClassifier(random_state=2023, n_jobs=-1)
    elif name == "CatBoost":
        return CatBoostClassifier(verbose=0, random_state=2023)
    else:
        raise ValueError("Unknown model name")

base_model_names = list(param_dists.keys())
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# 2) 도메인별 1st-layer 학습

best_domain_models = {}
domain_model_metrics = []

for domain, cols in domain_cols.items():
    print(f"\n===== [{domain}] 도메인 학습 ({len(cols)} vars) =====")
    Xtr, Xte = X_train[cols].copy(), X_test[cols].copy()
    oversampler = RandomOverSampler(random_state=42)

    best_auc, best_model, best_name, best_params = -np.inf, None, None, None

    for mdl_name in base_model_names:
        clf = make_base_clf(mdl_name)
        pipe = ImbPipeline([
            ('ros', oversampler),
            ('scaler', MinMaxScaler()),
            ('clf', clf)
        ])

        search = RandomizedSearchCV(
            estimator=pipe,
            param_distributions=param_dists[mdl_name],
            n_iter=12,
            cv=cv5,
            scoring='roc_auc',
            n_jobs=-1,
            random_state=42,
            verbose=0
        )
        search.fit(Xtr, y_train)
        mean_auc = search.best_score_

        print(f"  - {mdl_name:<18} best AUC (cv5): {mean_auc:.3f} | params: {search.best_params_}")

        # 🔹 Test-set 평가
        best_est = search.best_estimator_
        y_pred = best_est.predict(Xte)
        y_prob = best_est.predict_proba(Xte)[:, 1]

        domain_model_metrics.append({
            "Domain": domain,
            "Model": mdl_name,
            "CV-AUC": round(mean_auc, 3),
            "Accuracy": round(accuracy_score(y_test, y_pred), 3),
            "Precision": round(precision_score(y_test, y_pred), 3),
            "Recall": round(recall_score(y_test, y_pred), 3),
            "F1": round(f1_score(y_test, y_pred), 3),
            "AUROC": round(roc_auc_score(y_test, y_prob), 3),
            "PR-AUC": round(average_precision_score(y_test, y_prob), 3),
            "Balanced Accuracy": round(balanced_accuracy_score(y_test, y_pred), 3),
            "Best Params": search.best_params_
        })

        if mean_auc > best_auc:
            best_auc, best_model, best_name, best_params = mean_auc, best_est, mdl_name, search.best_params_

    best_domain_models[domain] = {
        "name": best_name,
        "pipeline": best_model,
        "cv_auc": best_auc,
        "params": best_params
    }
    print(f"▶ [{domain}] 선택 모델: {best_name}, CV-AUC={best_auc:.3f}")

# 결과 정리
domain_model_results = pd.DataFrame(domain_model_metrics)
domain_model_results = domain_model_results.sort_values(by=["Domain", "AUROC"], ascending=[True, False])
display(domain_model_results)
domain_model_results.to_excel("domain_model_all_results.xlsx", index=False)


===== [Bio] 도메인 학습 (3 vars) =====


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 4 is smaller than n_iter=12. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


  - Logistic Regression best AUC (cv5): 0.553 | params: {'clf__C': 1}
  - Random Forest      best AUC (cv5): 0.552 | params: {'clf__n_estimators': 100, 'clf__min_samples_split': 5, 'clf__min_samples_leaf': 1, 'clf__max_depth': 5}


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [05:29:40] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


  - XGBoost            best AUC (cv5): 0.552 | params: {'clf__n_estimators': 100, 'clf__max_depth': 3, 'clf__learning_rate': 0.1}
[LightGBM] [Info] Number of positive: 99735, number of negative: 99735
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002951 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 44
[LightGBM] [Info] Number of data points in the train set: 199470, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
  - LightGBM           best AUC (cv5): 0.550 | params: {'clf__n_estimators': 100, 'clf__max_depth': 10, 'clf__learning_rate': 0.05}


...

/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 99735, number of negative: 99735
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003763 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 175
[LightGBM] [Info] Number of data points in the train set: 199470, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
  - LightGBM           best AUC (cv5): 0.652 | params: {'clf__n_estimators': 100, 'clf__max_depth': 10, 'clf__learning_rate': 0.05}


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


  - CatBoost           best AUC (cv5): 0.652 | params: {'clf__n_estimators': 100, 'clf__learning_rate': 0.1, 'clf__depth': 7}
▶ [Lifestyle] 선택 모델: LightGBM, CV-AUC=0.652


,Domain,Model,CV-AUC,Accuracy,Precision,Recall,F1,AUROC,PR-AUC,Balanced Accuracy,Best Params
1,Bio,Random Forest,0.552,0.615,0.294,0.393,0.336,0.552,0.290,0.541,"{'clf__n_estimators': 100, 'clf__min_samples_s..."
2,Bio,XGBoost,0.552,0.602,0.291,0.421,0.344,0.552,0.290,0.541,"{'clf__n_estimators': 100, 'clf__max_depth': 3..."
4,Bio,CatBoost,0.553,0.591,0.288,0.440,0.348,0.552,0.290,0.541,"{'clf__n_estimators': 100, 'clf__learning_rate..."
0,Bio,Logistic Regression,0.553,0.527,0.275,0.554,0.367,0.551,0.290,0.536,{'clf__C': 1}
3,Bio,LightGBM,0.550,0.604,0.290,0.411,0.340,0.551,0.288,0.540,"{'clf__n_estimators': 100, 'clf__max_depth': 1..."
17,Lifestyle,XGBoost,0.652,0.629,0.348,0.569,0.432,0.658,0.410,0.609,"{'clf__n_estimators': 100, 'clf__max_depth': 5..."
18,Lifestyle,LightGBM,0.652,0.622,0.344,0.582,0.433,0.658,0.411,0.608,"{'clf__n_estimators': 100, 'clf__max_depth': 1..."
19,Lifestyle,CatBoost,0.652,0.624,0.346,0.578,0.433,0.658,0.409,0.609,"{'clf__n_estimators': 100, 'clf__learning_rate..."
16,Lifestyle,Random Forest,0.649,0.622,0.344,0.579,0.431,0.656,0.407,0.608,"{'clf__n_estimators': 300, 'clf__min_samples_s..."
15,Lifestyle,Logistic Regression,0.591,0.577,0.305,0.553,0.393,0.596,0.325,0.569,{'clf__C': 10}


In [ ]:
import joblib

tag = "Full_MICE_ensemble"

joblib.dump(best_domain_models, f"best_domain_models_{tag}.pkl")
joblib.dump(domain_model_results, f"domain_model_all_results_{tag}.pkl")

print("\n>>> 1st-layer 결과 저장 완료!")
print(f"- best_domain_models_{tag}.pkl")
print(f"- domain_model_all_results_{tag}.pkl")


>>> 1st-layer 결과 저장 완료!
- best_domain_models_Full_MICE_ensemble.pkl
- domain_model_all_results_Full_MICE_ensemble.pkl


In [ ]:
# 3) 2nd Layer용 OOF 예측 생성
from sklearn.model_selection import cross_val_predict

meta_train = pd.DataFrame(index=X_train.index)
meta_test  = pd.DataFrame(index=X_test.index)

for domain, info in best_domain_models.items():
    cols = domain_cols[domain]
    Xtr = X_train[cols].copy()
    Xte = X_test[cols].copy()

    pipe = info["pipeline"]

    # OOF predictions
    oof_pred = cross_val_predict(
        pipe,
        Xtr,
        y_train,
        method="predict_proba",
        cv=cv5,
        n_jobs=-1
    )[:, 1]

    # final test predictions
    pipe.fit(Xtr, y_train)
    test_pred = pipe.predict_proba(Xte)[:, 1]

    meta_train[f"{domain}_proba"] = oof_pred
    meta_test[f"{domain}_proba"]  = test_pred

print("\n>>> meta_train / meta_test 생성 완료!")
display(meta_train.head())

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:54:44] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Number of positive: 99735, number of negative: 99735
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002404 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 175
[LightGBM] [Info] Number of data points in the train set: 199470, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



>>> meta_train / meta_test 생성 완료!


,Bio_proba,Psych_proba,Social_proba,Lifestyle_proba
29321,0.569279,0.779151,0.625567,0.584305
131911,0.435799,0.450351,0.413661,0.413255
97716,0.473270,0.503884,0.349612,0.381488
11344,0.532245,0.519930,0.575963,0.404832
31372,0.452741,0.349466,0.450025,0.457384


In [ ]:
# 4) 2nd Layer (Meta: Logistic Regression)
meta_clf = LogisticRegression(max_iter=1000, solver='liblinear', random_state=42)
meta_clf.fit(meta_train, y_train)

y_pred_meta = meta_clf.predict(meta_test)
y_prob_meta = meta_clf.predict_proba(meta_test)[:, 1]

print("\n>>> 메타 모델 학습 완료!")

def summarize(y_true, y_pred, y_prob):
    return {
        "Accuracy": round(accuracy_score(y_true, y_pred), 3),
        "Precision": round(precision_score(y_true, y_pred), 3),
        "Recall": round(recall_score(y_true, y_pred), 3),
        "F1": round(f1_score(y_true, y_pred), 3),
        "AUROC": round(roc_auc_score(y_true, y_prob), 3),
        "PR-AUC": round(average_precision_score(y_true, y_prob), 3),
        "Balanced Accuracy": round(balanced_accuracy_score(y_true, y_pred), 3)
    }

stacking_results = []


>>> 메타 모델 학습 완료!


In [ ]:
# 5) 결과 요약 (도메인별 + 스태킹)
results = []

for domain, info in best_domain_models.items():
    Xte = X_test[domain_cols[domain]].copy()
    pipe = info["pipeline"]
    y_pred = pipe.predict(Xte)
    y_prob = pipe.predict_proba(Xte)[:, 1]
    res = summarize(y_test, y_pred, y_prob)
    res["Model"] = f"{domain} ({info['name']})"
    results.append(res)

res_meta = summarize(y_test, y_pred_meta, y_prob_meta)
res_meta["Model"] = "Stacking (Meta: Logistic)"
results.append(res_meta)

results_df = pd.DataFrame(results).sort_values(["AUROC", "F1"], ascending=False)
display(results_df)
results_df.to_excel("final_stacking_results.xlsx", index=False)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,Accuracy,Precision,Recall,F1,AUROC,PR-AUC,Balanced Accuracy,Model
4,0.765,0.604,0.154,0.245,0.706,0.456,0.560,Stacking (Meta: Logistic)
3,0.622,0.344,0.582,0.433,0.658,0.411,0.608,Lifestyle (LightGBM)
1,0.512,0.305,0.763,0.436,0.640,0.367,0.596,Psych (CatBoost)
2,0.595,0.313,0.535,0.395,0.604,0.329,0.574,Social (XGBoost)
0,0.591,0.288,0.440,0.348,0.552,0.290,0.541,Bio (CatBoost)


In [ ]:
# 신뢰구간 추가
from statsmodels.stats.proportion import proportion_confint

def calc_ci(metric_value, n, alpha=0.05):
    low, high = proportion_confint(metric_value * n, n, alpha=alpha, method='normal')
    return (round(low, 3), round(high, 3))

n_test = len(y_test)
ci_metrics = ["Accuracy","Precision","Recall","F1","AUROC","PR-AUC","Balanced Accuracy"]

# ---- Domain-level ----
domain_model_CI = domain_model_results.copy()
for metric in ci_metrics:
    domain_model_CI[f"{metric}_CI"] = domain_model_CI[metric].apply(lambda x: calc_ci(x, n_test))

domain_model_CI.to_excel("domain_model_all_results_with_CI.xlsx", index=False)
print(">>> domain_model_all_results_with_CI.xlsx 저장 완료!")

# ---- Stacking-level ----
results_df_CI = results_df.copy()
for metric in ci_metrics:
    results_df_CI[f"{metric}_CI"] = results_df_CI[metric].apply(lambda x: calc_ci(x, n_test))

results_df_CI.to_excel("final_stacking_results_with_CI.xlsx", index=False)
print(">>> final_stacking_results_with_CI.xlsx 저장 완료!")


>>> domain_model_all_results_with_CI.xlsx 저장 완료!
>>> final_stacking_results_with_CI.xlsx 저장 완료!


In [ ]:
tag = "Full_MICE_ensemble"

joblib.dump(meta_train, f"meta_train_{tag}.pkl")
joblib.dump(meta_test, f"meta_test_{tag}.pkl")
joblib.dump(meta_clf, f"meta_clf_{tag}.pkl")
joblib.dump(results_df, f"stacking_results_{tag}.pkl")
joblib.dump(results_df_CI, f"stacking_results_CI_{tag}.pkl")
joblib.dump(domain_model_CI, f"domain_model_all_results_CI_{tag}.pkl")

print("\n>>> 모든 결과 저장 완료! (CI 포함, 재학습 NO)")


>>> 모든 결과 저장 완료! (CI 포함, 재학습 NO)


최종적으로 생성되는 파일
> Domain-level (개별 모델)

domain_model_all_results.xlsx

domain_model_all_results_with_CI.xlsx

best_domain_models_Full_MICE_ensemble.pkl

domain_model_all_results_Full_MICE_ensemble.pkl

domain_model_all_results_CI_Full_MICE_ensemble.pkl

> Stacking-level

final_stacking_results.xlsx

final_stacking_results_with_CI.xlsx

stacking_results_Full_MICE_ensemble.pkl

stacking_results_CI_Full_MICE_ensemble.pkl

meta_train / meta_test / meta_clf 저장됨